In [1]:
%cd ../
import pandas as pd
from torch.utils.data import Dataset,DataLoader
import torch
import numpy as np
import ast
import torch.nn as nn
import torch.optim as optim
import random
from matplotlib import pyplot as plt
import os
import shutil
from datetime import datetime
import wandb
from utils import set_all_seed, load_exp2_dataset
from topk_mmoe import TopkMMoE
from trainer_and_evaluator import train_TopkMMoE, eval_TopkMMoE

C:\jupyter file\CMU_3\distributed ML\project



In [2]:
set_all_seed(42)
train_dataset, val_dataset, test_dataset = load_exp2_dataset()

Absolute Pearson correlation coefficient: 0.16661069841980908
P-value: 0.0


In [3]:
# 设定各种超参数，wandb日志名，日志存储路径，模型存储路径
train_params = {
    "batch_size": 256,
    "N_epochs": 50,
    "lr": 0.0001
}

# 仅设置dropout
model_params={
    "feature_dim": 114,
    "expert_dim": 32,
    "n_expert": 16,
    "n_activated_expert": 4,
    "n_task": 2,
    "sparse_load_balancing_loss_coef": 0,
    "olmo_load_balancing_loss_coef": 1e-5, # 采用OLMO的load-balancing loss
    "router_z_loss_coef": 1e-5, # 添加router-z-loss
    "gate_dropout": 0,
    "tower_dropout": 0,
    "expert_dropout": 0
}

model_name="exp2_topk_mmoe_olmo"
if not os.path.exists("model/"+model_name):
    os.makedirs("model/"+model_name) 

train_params_str = "_".join(f"{key}={value}" for key, value in train_params.items())
model_params_str = "_".join(f"{key}={value}" for key, value in model_params.items())
short_model_params_str = "_".join(f"{value}" for key, value in model_params.items())

wandb_name=model_name+":"+train_params_str+"_"+model_params_str

# 使用short_model_params_str是因为windows支持的最长文件名长度仅为260
bestmodel_save_dir=f"model/"+model_name+"/"+train_params_str+"_"+short_model_params_str 

print(wandb_name)
print(bestmodel_save_dir)

wandb.init(project="mmoe", name=wandb_name)

exp2_topk_mmoe_olmo:batch_size=256_N_epochs=50_lr=0.0001_feature_dim=114_expert_dim=32_n_expert=16_n_activated_expert=4_n_task=2_sparse_load_balancing_loss_coef=0_olmo_load_balancing_loss_coef=1e-05_router_z_loss_coef=1e-05_gate_dropout=0_tower_dropout=0_expert_dropout=0
model/exp2_topk_mmoe_olmo/batch_size=256_N_epochs=50_lr=0.0001_114_32_16_4_2_0_1e-05_1e-05_0_0_0


wandb: Currently logged in as: yuntaozh (zhengyuntao). Use `wandb login --relogin` to force relogin


In [4]:
mymodel = TopkMMoE(**model_params)

nParams = sum([p.nelement() for p in mymodel.parameters()])
print('Number of parameters: %d' % nParams)

nParams_in_mmoe=0
for name,p in mymodel.named_parameters():
    if name.startswith("sparse_mmoe"):
        nParams_in_mmoe=nParams_in_mmoe+p.nelement()
print('Number of parameters in SparseMMoE: %d' % nParams_in_mmoe)
# 相比于MMOE增加了w_noises和b_noises
print(f'Number of activated parameters in SparseMMoE: {int(nParams_in_mmoe*model_params["n_activated_expert"]/model_params["n_expert"])}')

Number of parameters: 45586
Number of parameters in SparseMMoE: 43584
Number of activated parameters in SparseMMoE: 10896


In [5]:
losses, val_losses, adam_batch_loss= train_TopkMMoE(mymodel,
                                           train_dataset,
                                           val_dataset,
                                           bestmodel_save_dir,
                                           **train_params)

Currently using device:cuda
Epoch=0,train_loss=0.8422713279724121,val_loss=0.5829333066940308
current epoch is the best so far. Saving model...
Epoch=1,train_loss=0.5383756756782532,val_loss=0.4854178726673126
current epoch is the best so far. Saving model...
Epoch=2,train_loss=0.46644696593284607,val_loss=0.4224870800971985
current epoch is the best so far. Saving model...
Epoch=3,train_loss=0.41165655851364136,val_loss=0.3869393467903137
current epoch is the best so far. Saving model...
Epoch=4,train_loss=0.36079922318458557,val_loss=0.3391623795032501
current epoch is the best so far. Saving model...
Epoch=5,train_loss=0.33329764008522034,val_loss=0.31514275074005127
current epoch is the best so far. Saving model...
Epoch=6,train_loss=0.32033002376556396,val_loss=0.31100720167160034
current epoch is the best so far. Saving model...
Epoch=7,train_loss=0.3092286288738251,val_loss=0.2971043288707733
current epoch is the best so far. Saving model...
Epoch=8,train_loss=0.2995950877666473

In [6]:
# load best model based on validation
mybestmodel = TopkMMoE(**model_params)
mybestmodel.load_state_dict(torch.load(bestmodel_save_dir))

<All keys matched successfully>

In [7]:
auc1, auc2=eval_TopkMMoE(mybestmodel, test_dataset)

AUC: 0.9764772042578831
AUC: 0.9915797080069078


In [8]:
wandb.finish()

load_balancing_loss,██▇▇▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▂▁▁▁▁▁
router_z_loss,▅▇▄▇▃▂▁▃▆▃▂▁▁▅▃▄▄▅▅▁▃▁▂▁▄▁▄▆▂▅▂█▂▁▁▄▆▄▂▆
task1_loss,▄█▄▃▃▂▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▃▃▂▂▂▂▂▂▂▂▂▂▂▃▂
task2_loss,█▆▄▂▄▂▂▂▃▂▂▂▂▂▂▂▂▃▁▂▁▂▃▁▁▁▂▃▂▁▂▁▁▂▂▂▂▁▁▂
task_0/expert_0_weight,▂▃▁▁▁▁▁▁▁▂▁█▅▄█▆▆▆▇▃▃▄▆▄▃▅▄▄▅▅▂▅▂▄▅▃▂▃▃▂
task_0/expert_10_weight,▁▁▁▂█▄▃▅▄▄▆▃▄▄▄▃▄▃▃▃▄▃▃▂▂▃▃▃▃▃▃▃▄▃▃▃▂▃▂▃
task_0/expert_11_weight,▁▁▄▄▄▅▇▇██▇▅▆▇▃▅█▇▅▄▇▆▅▄▄▅▅▄▅▄▅▅▂▅▅▅▅▄▇▄
task_0/expert_12_weight,▁▁▁▁▂▃▃█▆▆▆▅▆▆▄▅▆▆▅▆▄▅▅▅▆▆▅▆▅▅▄▆▆▅▆▆▅▇▅▆
task_0/expert_13_weight,▁▁▁▁▁▁▁▂▆▆▅▆▅▃▂▄▄▄▄▃▃▄▄▄▃▆▅▃▄▅▆▂█▄▄▂▅▃▅▆
task_0/expert_14_weight,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
task_0/expert_15_weight,▁▁▁▁▁▁█▁▂▃▂▁▂▂▂▂▂▂▂▂▂▂▂▃▂▁▂▂▂▂▂▂▂▂▂▂▂▃▂▃
